## 01 获取干员链接
存为文件`01-Name-Link.csv`

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

SKIP_LIST = ["可露希尔", "博士", "魏彦吾", "文月"]
DATA_PATH = "data/"
link_all_operator = "https://prts.wiki/w/%E5%B9%B2%E5%91%98%E7%BC%96%E5%8F%B7%E4%B8%80%E8%A7%88"

r = requests.get(link_all_operator)
r.encoding = "utf-8"
html = r.text

# 解析
soup = BeautifulSoup(html, "html.parser")

In [2]:
file_name_01 = "01-Name-Link.csv"
prefix = "https://prts.wiki"

# 寻找所有class="wikitable"的table
tables = []
for item in soup.find_all("table"):
    if item.get("class") == ["wikitable"]:
        tables.append(item)

# 寻找其中所有带有”herf“属性的标签，将该<a>标签的title记录进列表
list_all_operator_name = [] # 干员名称
list_all_operator_link = [] # 干员herf

for table in tables:
    # 定位到tbody
    tbody = table.find("tbody")
    # 在tbody中遍历所有tr，记录：herf、干员名称
    for tr in tbody.find_all("a"):
        name = tr.get("title")
        if name not in SKIP_LIST:
            list_all_operator_name.append(name)
            list_all_operator_link.append((prefix + tr.get("href")))

In [3]:
# 长度
len(list_all_operator_name), len(list_all_operator_link)

(293, 293)

In [4]:
# 抽样检测
list_all_operator_name[111], list_all_operator_link[111]

('玫拉', 'https://prts.wiki/w/%E7%8E%AB%E6%8B%89')

## 修正

In [5]:
# 将name中的特定几个值按下面的字典替换
dict_name = {
    "术髓": "Pith",
    "触痕": "Touch",
    "锋刃": "Sharp",
    "风暴眼": "Stormeye",
    }

for item in list_all_operator_name:
    if item in dict_name.keys():
        list_all_operator_name[list_all_operator_name.index(item)] = dict_name[item]

In [6]:
list_all_operator_name[0], list_all_operator_link[0]

('阿米娅', 'https://prts.wiki/w/%E9%98%BF%E7%B1%B3%E5%A8%85')

In [7]:
lost_name = ['预备干员-术师', '预备干员-后勤', '预备干员-狙击', '预备干员-近战', '阿米娅(近卫)', '预备干员-重装']
lost_link = ["https://prts.wiki/w/" + i for i in lost_name]

# 将lost加入 list_all_operator_name 后面
for i in range(len(lost_link)):
    list_all_operator_name.append(lost_name[i])
    list_all_operator_link.append(lost_link[i])

In [8]:
# 储存
df = pd.DataFrame({'name':list_all_operator_name, 'link':list_all_operator_link})
df.to_csv(DATA_PATH + file_name_01, index=False)